Testing PCAP file parisng

In [1]:
import sys
import logging
from scapy.all import *
from scapy.layers.l2 import Ether
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import time

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

In [ ]:
def read_pcap(pcap_file):
    try:
        packets = RawPcapReader(pcap_file)
    except FileNotFoundError:
        logger.error(f"PCAP file not found: {pcap_file}")
        sys.exit(1)
    except Scapy_Exception as e:
        logger.error(f"Error reading PCAP file: {e}")
        sys.exit(1)
    return packets


In [ ]:
def printable_timestamp(ts, resol):
    ts_sec = ts // resol
    ts_subsec = ts % resol
    ts_sec_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ts_sec))
    return '{}.{}'.format(ts_sec_str, ts_subsec)

In [54]:
def extract_packet_data(packets):
    packet_data = []
    flag = 0
    for (pkt_data, pkt_metadata,) in tqdm(packets, desc="Processing packets", unit="packet"):
        ether_pkt  = Ether(pkt_data)
       
        if 'type' not in ether_pkt.fields:
            # LLC frames will have 'len' instead of 'type'.
            # We disregard those
            continue

        if ether_pkt.type != 0x0800:
            # disregard non-IPv4 packets
            continue

        if ether_pkt.haslayer(IP):    
            packet = ether_pkt[IP]
    
            src_port = None
            dst_port = None
            src_ip = packet.src
            dst_ip = packet.dst
            pkg_len = packet.len
    
            if ether_pkt.haslayer(TCP):
               #packet_tcp = packet[TCP]
               src_port = int(packet.sport)
               dst_port = int(packet.dport)
               flag = int(packet.flags)
            
            if ether_pkt.haslayer(UDP):
               #packet_tcp = packet[UDP]
               src_port = int(packet.sport)
               dst_port = int(packet.dport)
               #flag = packet_tcp.flags
               
            protocol = packet.proto
    
            
            timestamp = pkt_metadata.sec + (pkt_metadata.usec)/1000000
            #if count == 0:
                #first_timestamp = timestamp
                #relative_timestamp = 0.0
            #else:
                #relative_timestamp = timestamp - first_timestamp
    
            
            pkt_size = pkt_metadata.caplen # Get packet size
                    
            packet_data.append({"src_ip": src_ip, "dst_ip": dst_ip, "protocol": protocol, "src_port": src_port,"dst_port": dst_port,"pkt_size": pkt_size,"flags":flag, "timestamp": str(round(timestamp, 6))})

    return pd.DataFrame(packet_data)

In [55]:
def protocol_name(number):
    protocol_dict = {1: 'ICMP', 6: 'TCP', 17: 'UDP'}
    return protocol_dict.get(number, f"Unknown({number})")

In [57]:
packets = read_pcap('local_dump.pcap')

In [58]:
df = extract_packet_data(packets)

Processing packets: 984packet [00:00, 2278.97packet/s]


In [59]:
df

,src_ip,dst_ip,protocol,src_port,dst_port,pkt_size,flags,timestamp
0,149.154.167.99,192.168.98.57,6,443,50788,60,2,1719997586.519905
1,52.112.164.167,192.168.98.57,17,3479,50012,148,2,1719997586.534998
2,52.112.164.167,192.168.98.57,17,3481,50058,114,2,1719997586.556402
3,52.112.164.167,192.168.98.57,17,3479,50012,148,2,1719997586.557581
4,52.112.164.167,192.168.98.57,17,3479,50012,173,2,1719997586.575281
...,...,...,...,...,...,...,...,...
884,52.112.164.167,192.168.98.57,17,3481,50056,166,2,1719997594.513788
885,192.168.98.57,52.112.164.167,17,50012,3479,158,2,1719997594.533349
886,192.168.98.57,149.154.167.99,6,50788,443,221,2,1719997594.534322
887,149.154.167.99,192.168.98.57,6,443,50788,60,2,1719997594.545343


Straem packeges test

In [3]:
def packet_callback(packet):
    if packet[IP].proto == 6:  # ICMP protocol
        print(f"ICMP packet from {packet[IP].src}")

In [4]:
sniff(prn=packet_callback, store=0)

IndexError: Layer [IP] not found